In [1]:
import os
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np # linear algebra
import pandas as pd 
import os
import keras


#add dataset
DataPath = "/content/drive/MyDrive/Colab Notebooks/projet_images _filaments/tiffs_easy_smaller_dataset_segmentation/"
images = []
masks = []
for dirname, _, filenames in os.walk(DataPath):
    for filename in filenames:
        if '_mask'in filename:
            
            masks.append(filename)
            images.append(filename.replace('_mask', ''))
print(len(images), len(masks)) 
imagePath_df = pd.DataFrame({'images': images, 'masks': masks})

def print_imShape():
    idx = np.random.randint(0, len(imagePath_df))
    
    imagePath = os.path.join(DataPath, imagePath_df['images'].iloc[idx])
    maskPath = os.path.join(DataPath, imagePath_df['masks'].iloc[idx])
    
    image = cv2.imread(imagePath)
    mask = cv2.imread(maskPath)
    
    print(image.shape, mask.shape)
for i in range(5):
    print_imShape()

imagePath_df['image-path'] = DataPath + '/' + imagePath_df['images']
imagePath_df['mask-path'] = DataPath + '/' + imagePath_df['masks'] 




df_train, df_test = train_test_split(imagePath_df,test_size = 0.2)
df_test, df_val = train_test_split(df_test,test_size = 0.5)
print(df_train.values.shape)
print(df_val.values.shape)
print(df_test.values.shape)


#Set Parameters
EPOCHS = 150
BATCH_SIZE = 2
learning_rate = 1e-3
im_width = 666
im_height = 666


def train_generator(data_frame, batch_size, aug_dict,
        image_color_mode="rgb",
        mask_color_mode="grayscale",
        image_save_prefix="image",
        mask_save_prefix="mask",
        save_to_dir=None,
        target_size=(im_width,im_height),
        seed=1):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        x_col = "image-path",
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)

    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        x_col = "mask-path",
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)

    train_gen = zip(image_generator, mask_generator)
    
    for (img, mask) in train_gen:
        img, mask = adjust_data(img, mask)
        yield (img,mask)

def adjust_data(img,mask):
    img = img / 255
    mask = mask / 255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    
    return (img, mask)


smooth=100

def dice_coef(y_true, y_pred):
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)
    And=K.sum(y_truef* y_predf)
    return((2* And + smooth) / (K.sum(y_truef) + K.sum(y_predf) + smooth))

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

def jac_distance(y_true, y_pred):
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)

    return - iou(y_true, y_pred)

#model unet


def unet(input_size=(im_width,im_height,3)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (9, 9), padding='same')(inputs)
    bn1 = Activation('relu')(conv1)
    conv1 = Conv2D(32, (9, 9), padding='same')(bn1)
    bn1 = BatchNormalization(axis=3)(conv1)
    bn1 = Activation('relu')(bn1)
    conv1 = Conv2D(32, (9, 9), padding='same')(bn1)
    bn1 = BatchNormalization(axis=3)(conv1)
    bn1 = Activation('relu')(bn1)
    pool1 = MaxPooling2D(pool_size=(3, 3))(bn1)
    pool1 = Dropout(0.1)(pool1)

    conv2 = Conv2D(64, (9, 9), padding='same')(pool1)
    bn2 = Activation('relu')(conv2)
    conv2 = Conv2D(64, (9, 9), padding='same')(bn2)
    bn2 = BatchNormalization(axis=3)(conv2)
    bn2 = Activation('relu')(bn2)
    conv2 = Conv2D(64, (9, 9), padding='same')(bn2)
    bn2 = BatchNormalization(axis=3)(conv2)
    bn2 = Activation('relu')(bn2)
    pool2 = MaxPooling2D(pool_size=(3, 3))(bn2)
    pool2 = Dropout(0.1)(pool2)
    

    conv3 = Conv2D(128, (9, 9), padding='same')(pool2)
    bn3 = Activation('relu')(conv3)
    conv3 = Conv2D(128, (9, 9), padding='same')(bn3)
    bn3 = BatchNormalization(axis=3)(conv3)
    bn3 = Activation('relu')(bn3)
    conv3 = Conv2D(128, (9, 9), padding='same')(bn3)
    bn3 = BatchNormalization(axis=3)(conv3)
    bn3 = Activation('relu')(bn3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)
    pool3 = Dropout(0.1)(pool3)
    

    

    conv4 = Conv2D(256, (9, 9), padding='same')(pool3)
    bn4 = Activation('relu')(conv4)
    conv4 = Conv2D(256, (9, 9), padding='same')(bn4)
    bn4 = BatchNormalization(axis=3)(conv4)
    bn4 = Activation('relu')(bn4)
    conv4 = Conv2D(256, (9, 9), padding='same')(bn4)
    bn4 = BatchNormalization(axis=3)(conv4)
    bn4 = Activation('relu')(bn4)
    pool4 = MaxPooling2D(pool_size=(1, 1))(bn4)
    pool4 = Dropout(0.1)(pool4)
    

    conv5 = Conv2D(512, (9, 9), padding='same')(pool4)
    bn5 = Activation('relu')(conv5)
    conv5 = Conv2D(512, (9, 9), padding='same')(bn5)
    bn5 = BatchNormalization(axis=3)(conv5)
    bn5 = Activation('relu')(bn5)
    conv5 = Conv2D(512, (9, 9), padding='same')(bn5)
    bn5 = BatchNormalization(axis=3)(conv5)
    bn5 = Activation('relu')(bn5)

    up6 = concatenate([Conv2DTranspose(256, (9, 9), strides=(1, 1), padding='same')(bn5), conv4], axis=3)
    conv6 = Conv2D(256, (9, 9), padding='same')(up6)
    bn6 = Activation('relu')(conv6)
    conv6 = Conv2D(256, (9, 9), padding='same')(bn6)
    bn6 = BatchNormalization(axis=3)(conv6)
    bn6 = Activation('relu')(bn6)

    up7 = concatenate([Conv2DTranspose(128, (9, 9), strides=(2, 2), padding='same')(bn6), conv3], axis=3)
    up7 = Dropout(0.1)(up7)
    conv7 = Conv2D(128, (9, 9), padding='same')(up7)
    bn7 = Activation('relu')(conv7)
    conv7 = Conv2D(128, (9, 9), padding='same')(bn7)
    bn7 = BatchNormalization(axis=3)(conv7)
    bn7 = Activation('relu')(bn7)

    up8 = concatenate([Conv2DTranspose(64, (9, 9), strides=(3, 3), padding='same')(bn7), conv2], axis=3)
    up8 = Dropout(0.1)(up8)
    conv8 = Conv2D(64, (9, 9), padding='same')(up8)
    bn8 = Activation('relu')(conv8)
    conv8 = Conv2D(64, (9, 9), padding='same')(bn8)
    bn8 = BatchNormalization(axis=3)(conv8)
    bn8 = Activation('relu')(bn8)

    up9 = concatenate([Conv2DTranspose(32, (9, 9), strides=(3, 3), padding='same')(bn8), conv1], axis=3)
    up9 = Dropout(0.1)(up9)
    conv9 = Conv2D(32, (9, 9), padding='same')(up9)
    bn9 = Activation('relu')(conv9)
    conv9 = Conv2D(32, (9, 9), padding='same')(bn9)
    bn9 = BatchNormalization(axis=3)(conv9)
    bn9 = Activation('relu')(bn9)

    

    

    

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(bn9)

    return Model(inputs=[inputs], outputs=[conv10])
model = unet()
model.summary()




#weighted_bincrossentropy
def weighted_bincrossentropy(true, pred, weight_zero = 0.25, weight_one = 1):
    """
    Calculates weighted binary cross entropy. The weights are fixed.
        
    This can be useful for unbalanced catagories.
    
    Adjust the weights here depending on what is required.
    
    For example if there are 10x as many positive classes as negative classes,
        if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives 
        will be penalize 10 times as much as false negatives.
    """
  
    # calculate the binary cross entropy
    bin_crossentropy = keras.backend.binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weighted_bin_crossentropy = weights * bin_crossentropy 

    return keras.backend.mean(weighted_bin_crossentropy)

#parametrer model
train_generator_args = dict(rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest')
train_gen = train_generator(df_train, BATCH_SIZE,
                                train_generator_args,
                                target_size=(im_height, im_width)
                                
                                )
test_gener = train_generator(df_val, BATCH_SIZE,
                                dict(),
                                target_size=(im_height, im_width))
model = unet(input_size=(im_width,im_height, 3))

decay_rate = learning_rate / EPOCHS
opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)
model.compile(optimizer=opt, loss=weighted_bincrossentropy, metrics=["accuracy", iou, dice_coef])

#trining model
callbacks = [ModelCheckpoint('unet_model_report_666_seg_easy_smaller_3_32f_150epoch.hdf5', verbose=1, save_best_only=True)]

history = model.fit(train_gen,
                    steps_per_epoch=len(df_train) / BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=callbacks,
                    validation_data = test_gener,
                    validation_steps=len(df_val) / BATCH_SIZE
                    )



a = history.history

list_traindice = a['accuracy']
list_testdice = a['val_accuracy']

list_trainjaccard = a['iou']
list_testjaccard = a['val_iou']

list_trainloss = a['loss']
list_testloss = a['val_loss']
plt.figure(1)
plt.plot(list_testloss, 'b-')
plt.plot(list_trainloss,'r-')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.title('loss graph', fontsize = 15)
plt.figure(2)
plt.plot(list_traindice, 'r-')
plt.plot(list_testdice, 'b-')
plt.xlabel('iteration')
plt.ylabel('accuracy')
plt.title('accuracy graph', fontsize = 15)
plt.show()


# load the best model
model.load_weights('./omar1/unet_model_report_666_seg_easy_smaller_3_32f_150epoch.hdf5')
#evaulate model
eval_results = model.evaluate(test_gener, steps=len(df_test) / BATCH_SIZE, verbose=1)


#test model
for i in range(23):
    idx = np.random.randint(0, len(df_test))
    
    imagePath = os.path.join(DataPath,  df_test['images'].iloc[idx])
    maskPath = os.path.join(DataPath, df_test['masks'].iloc[idx])
    
    image = cv2.imread(imagePath)
    mask = cv2.imread(maskPath)
    
    img = cv2.resize(image ,(im_width, im_width),1)
    img = img / 255
    mask=mask/255
    #mask = numpy.array(mask)
    img = img[np.newaxis, :, :, :]
    pred=model.predict(img)
    pred=(np.squeeze(pred))
    fig, axs =plt.subplots(1,3, figsize=[13,15])
    
    axs[0].imshow(np.squeeze(img))
    axs[0].set_title('noisy')
    axs[1].imshow(mask)
    axs[1].set_title('binary')
    
    axs[2].imshow(pred,cmap="gray")
    axs[2].set_title('Prediction')
    axs[0].grid(False)
    axs[1].grid(False)
    axs[2].grid(False)
    plt.show()



501 501
(666, 666, 3) (666, 666, 3)
(666, 666, 3) (666, 666, 3)
(666, 666, 3) (666, 666, 3)
(666, 666, 3) (666, 666, 3)
(666, 666, 3) (666, 666, 3)
(400, 4)
(51, 4)
(50, 4)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 666, 666, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 666, 666, 32  7808        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 activation (Activat

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Found 400 validated image filenames.
Found 400 validated image filenames.
Epoch 1/150
200/200 [==============================] - ETA: 0s - loss: 0.0837 - accuracy: 0.9335 - iou: 0.1293 - dice_coef: 0.2263Found 51 validated image filenames.
Found 51 validated image filenames.

Epoch 1: val_loss improved from inf to 0.07559, saving model to unet_model_report_666_seg_easy_smaller_3_32f_150epoch.hdf5
200/200 [==============================] - 454s 2s/step - loss: 0.0837 - accuracy: 0.9335 - iou: 0.1293 - dice_coef: 0.2263 - val_loss: 0.0756 - val_accuracy: 0.9650 - val_iou: 0.1143 - val_dice_coef: 0.2043
Epoch 2/150
 38/200 [====>.........................] - ETA: 5:22 - loss: 0.0490 - accuracy: 0.9518 - iou: 0.2060 - dice_coef: 0.3411

KeyboardInterrupt: ignored